In [1]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('helper_functions/')
# import util as util
import util

In [2]:
from pymongo import MongoClient

client = MongoClient()

In [3]:
db = client.airbnb


print('The list of all databases currently in the MongoDB client is:')
print(client.list_database_names())

print('\nThe list of all collections in the airbnb database is:')
print(db.list_collection_names())

The list of all databases currently in the MongoDB client is:
['admin', 'airbnb', 'config', 'local', 'test']

The list of all collections in the airbnb database is:
['listings', 'listings_with_calendar', 'reviews', 'listings_with_reviews_and_cal', 'listings_with_reviews_m', 'calendar']


In [5]:
dtype = {"listing_id": str, "date": str, "available": str, 
        "price": str, "adjusted_price": str}
calendar_df = pd.read_csv('Data/calendar.csv',dtype=dtype, keep_default_na = False)

C:\Users\Reason\AppData\Local\Temp\ipykernel_22016\199888413.py:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar_df = pd.read_csv('Data/calendar.csv',dtype=dtype, keep_default_na = False)


In [6]:
#First few views
print('The datatypes for the fields of df are:')
print(calendar_df.dtypes)

print('\nThe first few rows of df are:')
print(calendar_df.head())

The datatypes for the fields of df are:
listing_id        object
date              object
available         object
price             object
adjusted_price    object
minimum_nights    object
maximum_nights    object
dtype: object

The first few rows of df are:
  listing_id        date available    price adjusted_price minimum_nights  \
0     144087  2024-02-10         t  $259.00                            30   
1     144087  2024-02-11         t  $259.00                            30   
2     144087  2024-02-12         t  $259.00                            30   
3     144087  2024-02-13         t  $259.00                            30   
4     144087  2024-02-14         t  $259.00                            30   

  maximum_nights  
0            365  
1            365  
2            365  
3            365  
4            365  


In [7]:
importlib.reload(util)
calendar_df['date'] = calendar_df['date'].apply(util.convert_date_str_to_datetime)
calendar_df['available'] = calendar_df['available'].apply(util.convert_tf_to_boolean)

In [8]:
calendar_df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,144087,2024-02-10,True,$259.00,,30,365
1,144087,2024-02-11,True,$259.00,,30,365
2,144087,2024-02-12,True,$259.00,,30,365
3,144087,2024-02-13,True,$259.00,,30,365
4,144087,2024-02-14,True,$259.00,,30,365


In [9]:
calendar_df['price'] = pd.to_numeric(calendar_df['price'].apply(lambda x: x.replace('$', '').replace(',', '')))

In [10]:
print(type(calendar_df.loc[0,'price']))
print(calendar_df.head())


<class 'numpy.float64'>
  listing_id       date  available  price adjusted_price minimum_nights  \
0     144087 2024-02-10       True  259.0                            30   
1     144087 2024-02-11       True  259.0                            30   
2     144087 2024-02-12       True  259.0                            30   
3     144087 2024-02-13       True  259.0                            30   
4     144087 2024-02-14       True  259.0                            30   

  maximum_nights  
0            365  
1            365  
2            365  
3            365  
4            365  


In [11]:
time1 = datetime.now()
calendar_dict = calendar_df.to_dict('records')
time2 = datetime.now()
print(f'Time to perform this operation was {util.time_diff(time1,time2)} hms.')

Time to perform this operation was 0:01:15.006441 hms.


In [20]:
# The following empties out listings_with_cal; useful if making a fresh start
db.calendar.drop()

print(len(calendar_dict))

time1 = datetime.now()
result = db.calendar.insert_many(calendar_dict)
time2 = datetime.now()
print(f'\nTime to perform this operation was {util.time_diff(time1,time2)} seconds.')
# between about 2 and 4 minutes

print(f'\nNumber of docs in db.calendar is {db.calendar.count_documents({})}')

print()
outdocs = []
for o in result.inserted_ids[-5:]:
    outdocs.append(db.calendar.find_one({ '_id': o}))
pprint.pp(outdocs)

14299870

Time to perform this operation was 0:02:50.862937 seconds.

Number of docs in db.calendar is 14299870

[{'_id': ObjectId('6664e668b7602debbc665006'),
  'listing_id': '1081185973600372871',
  'date': datetime.datetime(2025, 1, 31, 0, 0),
  'available': False,
  'price': 160.0,
  'adjusted_price': '',
  'minimum_nights': 30,
  'maximum_nights': 365},
 {'_id': ObjectId('6664e668b7602debbc665007'),
  'listing_id': '1081185973600372871',
  'date': datetime.datetime(2025, 2, 1, 0, 0),
  'available': False,
  'price': 160.0,
  'adjusted_price': '',
  'minimum_nights': 30,
  'maximum_nights': 365},
 {'_id': ObjectId('6664e668b7602debbc665008'),
  'listing_id': '1081185973600372871',
  'date': datetime.datetime(2025, 2, 2, 0, 0),
  'available': False,
  'price': 160.0,
  'adjusted_price': '',
  'minimum_nights': 30,
  'maximum_nights': 365},
 {'_id': ObjectId('6664e668b7602debbc665009'),
  'listing_id': '1081185973600372871',
  'date': datetime.datetime(2025, 2, 3, 0, 0),
  'available

In [21]:
#Double Checking 
print(len(calendar_dict))
print(db.calendar.count_documents({}))

14299870
14299870


In [ ]:
db.listings_with_calendar.drop()

pipeline = [
    {
        '$group': {
            '_id': '$listing_id',
            'average_price': {'$avg': '$price'},
            'first_available_date': {'$min': '$date'},
            'last_available_date': {'$max': '$date'},
            'dates_list': {
                '$push': {
                    'date': '$date',
                    'available': '$available',
                    'price': '$price',
                    'minimum_nights': '$minimum_nights',
                    'maximum_nights': '$maximum_nights'
                }
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'id': '$_id',
            'average_price': 1,
            'first_available_date': 1,
            'last_available_date': 1,
            'dates_list': 1
        }
    },
    {
        '$addFields': {
            '_id': '$id'
        }
    },
    {
        '$out': 'listings_with_calendar'
    }
]


test1 = db.calendar.aggregate(pipeline)
print(db.list_collection_names())


In [25]:
print(db.list_collection_names())
print(db.listings_with_calendar.count_documents({}))

['listings', 'listings_with_calendar', 'calendar']
39201


In [ ]:
pprint.pp(db.listings_with_calendar.find_one({'id': '10000070'}))

In [33]:
print(db.listings_with_calendar.count_documents({}))

39201


In [ ]:
importlib.reload(util)
doc1 = db.listings_with_calendar.find_one({'id': '11194693'})

pprint.pp(util.convert_lwc_to_json(doc1))

In [12]:
cursor_docs = db.listings_with_calendar.find({'_id': {'$regex' : '^1000'}})

#docs = [doc for doc in cursor_docs]

In [13]:
output = []

for doc in cursor_docs:
    output.append(util.convert_lwc_to_json(doc))

print(len(output))

28


In [14]:
importlib.reload(util)

dir = 'CHECK'
filename = 'listings_with_calendar_subset.json'
util.write_dict_to_dir_json(output, dir, filename)